In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae') 

C:\Users\ayool\anaconda3\envs\data\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df['QScore']

0        3A
1        3A
2        3A
3        3A
4        3A
         ..
72181    3A
72182    3A
72183    3A
72184    3A
72185    3A
Name: QScore, Length: 72186, dtype: object

In [4]:
#check distribution of target variable 
df[ 'QScore' ].value_counts() 

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [5]:
df.isna().sum() 

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.isna().sum() 

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [8]:
#An obvious change in our target variable after removing the missing values is that there are only three classes left #and from the distribution of the 3 classes, we can see that 
#there is an obvious imbalance between the classes. There are methods that can be applied to handle this imbalance such as oversampling and undersampling. 
#Oversampling involves increasing the number of instances in the class with fewer instances 
#while undersampling #involves reducing the data points in the class with more instances. 
#For now, we will convert this to a binary classification problem by combining class '2A' and '1A'. 
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' ) 
df.QScore.value_counts() 
 

3A    51473
2A      240
Name: QScore, dtype: int64

In [9]:
df_2A = df[df.QScore== '2A' ] 
df_3A = df[df.QScore== '3A' ].sample( 350 ) 
data_df = df_2A.append(df_3A) 

In [10]:
df_2A

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
1536,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,2.258528e-02,2.998367e-02,0.000000e+00,1.119497e+00,2A
1537,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,9.171000e+05,1.217520e+06,0.000000e+00,4.545842e+07,2A
1538,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,7.947991e-03,2.924496e-02,0.000000e+00,5.301590e-01,2A
1539,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,3.227369e+05,1.187524e+06,0.000000e+00,2.152769e+07,2A
1540,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,1.472910e-02,2.924496e-02,1.391455e+00,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
65469,Ukraine,2016,230,BiocapTotGHA,8.966961e+07,6.457412e+06,19268322.04,6.920056e+06,3.952345e+06,0.000000e+00,1.262677e+08,2A
65470,Ukraine,2016,230,EFConsPerCap,1.155004e+00,1.081601e-02,0.149119494,5.150234e-02,8.893945e-02,1.453137e+00,2.908519e+00,2A
65471,Ukraine,2016,230,EFConsTotGHA,5.132678e+07,4.806484e+05,6626661.768,2.288692e+06,3.952345e+06,6.457538e+07,1.292505e+08,2A
65472,Ukraine,2016,230,EFProdPerCap,2.017831e+00,5.768143e-03,0.216720935,3.223480e-03,8.893945e-02,1.665767e+00,3.998251e+00,2A


In [11]:
import sklearn.utils 
data_df = sklearn.utils.shuffle(data_df) 
data_df = data_df.reset_index(drop= True ) 
data_df.shape 
data_df.QScore.value_counts() 


3A    350
2A    240
Name: QScore, dtype: int64

In [12]:
#more preprocessing 
data_df = data_df.drop(columns=[ 'country_code' , 'country' , 'year' ]) 
X = data_df.drop(columns= 'QScore' ) 
y = data_df[ 'QScore' ] 

In [13]:
#split the data into training and testing sets 
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 ) 
y_train.value_counts() 

3A    246
2A    167
Name: QScore, dtype: int64

In [14]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this. 
#encode categorical variable 
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() 
x_train.record = encoder.fit_transform(x_train.record) 
x_test.record = encoder.transform(x_test.record) 

In [15]:
pip install imblearn

ERROR: Could not find a version that satisfies the requirement scikit-learn>=1.1.0 (from imbalanced-learn->imblearn) (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0b1, 0.15.0b2, 0.15.0, 0.15.1, 0.15.2, 0.16b1, 0.16.0, 0.16.1, 0.17b1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19b2, 0.19.0, 0.19.1, 0.19.2, 0.20rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21rc2, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.22rc2.post1, 0.22rc3, 0.22, 0.22.1, 0.22.2, 0.22.2.post1, 0.23.0rc1, 0.23.0, 0.23.1, 0.23.2, 0.24.dev0, 0.24.0rc1, 0.24.0, 0.24.1, 0.24.2, 1.0rc1, 1.0rc2, 1.0, 1.0.1, 1.0.2)
ERROR: No matching distribution found for scikit-learn>=1.1.0 (from imbalanced-learn->imblearn)



  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)


In [16]:
import imblearn 
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state= 1 ) 
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train) 

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler() 
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=[ 'record' ])) 
normalised_train_df = pd.DataFrame(normalised_train_df, 
columns=x_train_balanced.drop(columns=[ 'record' ]).columns) 
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ] 
x_test = x_test.reset_index(drop= True ) 
normalised_test_df = scaler.transform(x_test.drop(columns=[ 'record' ])) 
normalised_test_df = pd.DataFrame(normalised_test_df, 
columns=x_test.drop(columns=[ 'record' ]).columns) 
normalised_test_df[ 'record' ] = x_test[ 'record' ] 

In [ ]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression 
log_reg = LogisticRegression() 
log_reg.fit(normalised_train_df, y_balanced) 
#returns 
LogisticRegression(C= 1.0 , class_weight= None , dual= False , fit_intercept= True , 
 intercept_scaling= 1 , l1_ratio= None , max_iter= 100 , 
 multi_class= 'auto' , n_jobs= None , penalty= 'l2' , 
 random_state= None , solver= 'lbfgs' , tol= 0.0001 , verbose= 0 , 
 warm_start= False )